In [ ]:
# Reference: https://github.com/geopandas/geopandas/issues/1816#issuecomment-1003093329
# conda install -c conda-forge geopandas pygeos
# Reference: https://pyrosm.readthedocs.io/en/latest/installation.html
# pip install pyrosm

In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
import geopandas as gpd

## Data Parsing

### OSM Data

In [4]:
import pyrosm
from pyrosm.data import sources as get_osm_data_sources
from pyrosm import get_data as get_osm_data_as_pbf

In [5]:
fp = get_osm_data_as_pbf("Zambia") # access Jan 19, 2022 # if data of a later date is needed, use parameter to refresh

In [6]:
osm = pyrosm.OSM(fp)

In [46]:
driving_network = osm.get_network(network_type="driving")

# np.round(driving_network.isnull().mean().sort_values(ascending=True),2)

# Reference: https://wiki.openstreetmap.org/wiki/Key:highway
# major_driving_network = driving_network[driving_network['highway'].isin(['trunk', 'primary', 'secondary', 'tertiary', 'road', 'track', 'trunk_link', 'primary_link', 'secondary_link', 'tertiary_link'])]
driving_network = driving_network.dropna(axis=1)
driving_network.to_csv('./data/driving_network.csv',index=False)

driving_network.head(2)

In [62]:
buildings = osm.get_buildings()

# np.round(buildings.isnull().mean().sort_values(ascending=True),5)

buildings = buildings.dropna(axis=1)
buildings.to_csv('./data/buildings.csv',index=False)

buildings.head(2)

In [64]:
custom_filter = {'amenity': True, "shop": True}
pois = osm.get_pois(custom_filter=custom_filter)
pois["poi_type"] = pois["amenity"].fillna(pois["shop"])

/opt/homebrew/Caskroom/miniforge/base/envs/geo_utils_env/lib/python3.9/site-packages/pyrosm/pyrosm.py:576: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  gdf = get_poi_data(


In [96]:
# pois['geometry_type'] = pois.geom_type
# # pois.loc[pois['lon'].notnull(),'geometry_type'].value_counts()
# # pois.loc[pois['lon'].isnull(),'geometry_type'].value_counts()
pois['geometry_centroid'] = pois.centroid
pois = pois.set_geometry('geometry_centroid', drop=True)

In [123]:
# np.round(pois.isnull().mean().sort_values(ascending=True),5)
pois = pois.dropna(axis=1)
pois.to_csv('./data/pois.csv', index=False)

In [129]:
landuse = osm.get_landuse()

# np.round(landuse.isnull().mean().sort_values(ascending=True),5)
landuse = landuse.dropna(axis=1)
landuse.to_csv('./data/landuse.csv', index=False)

/opt/homebrew/Caskroom/miniforge/base/envs/geo_utils_env/lib/python3.9/site-packages/pyrosm/pyrosm.py:323: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  gdf = get_landuse_data(


In [22]:
pd.read_csv('./data/landuse.csv').head(2)

,id,version,timestamp,landuse,geometry,osm_type
0,2776917403,2,1522987582,conservation,POINT (31.704246520996094 -13.202841758728027),node
1,3893207581,1,1450176844,plant_nursery,POINT (27.549104690551758 -16.027915954589844),node


In [1]:
# natural = osm.get_natural()
## Seems to have out of memory issue with natural layer, worth investigating

### Facebook Road Data

https://github.com/facebookmicrosites/Open-Mapping-At-Facebook/wiki/Available-Countries

In [5]:
gdf = gpd.read_file('./data/ZM_mapwithai_road_data.gpkg')

In [7]:
gdf.highway_tag.value_counts()

track             1163592
residential        506087
unclassified        11204
path                 8100
service               533
tertiary              215
footway                92
secondary              28
trunk                  13
primary                 7
living_street           3
secondary_link          2
pedestrian              2
primary_link            1
trunk_link              1
Name: highway_tag, dtype: int64

In [8]:
df = pd.read_csv('./data/driving_network.csv')

In [13]:
network_ pd.merge(gdf.highway_tag.value_counts().rename('facebook'),df.highway.value_counts().rename('osm'), left_index=True, right_index=True, how='outer')

,facebook,osm
FARM,NaN,1.0
footway,92.0,1.0
living_street,3.0,4.0
path,8100.0,34.0
pedestrian,2.0,NaN
primary,7.0,1031.0
primary_link,1.0,57.0
residential,506087.0,74041.0
rest_area,NaN,1.0
road,NaN,136.0
